In [4]:
import operator
from typing import Annotated, List, TypedDict, Literal
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI



In [2]:
# Load API key
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

if not openai_api_key:
    raise ValueError("OPENAI_API_KEY not found!")

print("✅ API key loaded")

✅ API key loaded


In [3]:
# Initialize LLM
llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=openai_api_key
)

print(f"✅ LLM initialized: {llm.model_name}")

✅ LLM initialized: gpt-4o-mini


In [5]:
# State for the Hybrid Agent
class HybridState(TypedDict):
    input: str
    plan: List[str]
    current_step: int
    results: Annotated[List[str], operator.add]
    
    # Reflection fields
    draft: str
    critique: str
    iterations: int
    final_output: str

MAX_REFLECTIONS = 2

In [6]:
# Node 1: Planner
def planner(state: HybridState) -> dict:
    """Create a step-by-step plan."""
    prompt = f"""Create a step-by-step plan for this task:

Task: {state['input']}

Return a numbered list of concrete steps. Keep it simple (3-5 steps)."""
    
    response = llm.invoke([HumanMessage(content=prompt)])
    
    # Parse steps (simple parsing)
    lines = response.content.split('\n')
    steps = [line.strip() for line in lines if line.strip() and any(char.isdigit() for char in line[:3])]
    
    print(f"\n📋 PLAN CREATED:")
    for step in steps:
        print(f"  {step}")
    print()
    
    return {"plan": steps, "current_step": 0, "results": []}

# Node 2: Executor
def executor(state: HybridState) -> dict:
    """Execute the current step."""
    if state["current_step"] >= len(state["plan"]):
        # All steps done
        return {}
    
    current_step = state["plan"][state["current_step"]]
    
    print(f"⚙️ Executing: {current_step}")
    
    # Execute step (simplified - just use LLM)
    prompt = f"""Previous results: {state.get('results', [])}\n\nExecute this step: {current_step}"""
    response = llm.invoke([HumanMessage(content=prompt)])
    
    result = f"Step {state['current_step'] + 1} result: {response.content}"
    print(result)
    print(f"✓ Done\n")
    
    return {
        "results": [result],
        "current_step": state["current_step"] + 1
    }

# --- REFLECTION SUITE ---
def generator(state: HybridState):
    """Generates an initial draft from execution results"""
    print("Generating initial draft from execution results...")
    prompt = f"""Combine these research results into initial draft:
Search Results: {state['results']}
Task: {state['input']}

Write a clear, concise and accurate draft from the results"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content, "iterations": 0}

def critic(state: HybridState) -> dict:
    """Assess the draft against the original task"""
    prompt = f"""Critique this summary against the original task:
Draft: {state['draft']}
Task: {state['input']}
Critique this draft. Does it fulfil the task requirements? What could be improved?
If it's excellent, say "APPROVED".
Otherwise, provide specific improvements needed."""
    print("Critiquing draft...")
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"critique": response.content}

def refiner(state: HybridState):
    """Improves the draft based on critique."""
    print(f"♻️ Refining draft (Attempt {state['iterations'] + 1})...")
    prompt = f"""Improve this draft based on: 
Original Task: {state['input']}
Critique: {state['critique']}
Draft: {state['draft']}"""
    response = llm.invoke([HumanMessage(content=prompt)])
    return {"draft": response.content, "iterations": state["iterations"] + 1}

def finalizer(state: HybridState) -> dict:
    """Set final output."""
    print("\n✅ Reflection complete!\n")
    return {"final_output": state["draft"]}

print("✅ Reflection nodes defined")

✅ Reflection nodes defined


In [7]:
def should_execute(state: HybridState):
    """Decide if more steps to execute."""
    if state["current_step"] < len(state["plan"]):
        return "executor"
    return "generator"

def should_refine(state: HybridState):
    """Decide if we need more refinement."""
    if "APPROVED" in state["critique"].upper() or state["iterations"] >= MAX_REFLECTIONS:
        return "finalizer"
    return "refiner"

# Build Graph
builder = StateGraph(HybridState)

builder.add_node("planner", planner)
builder.add_node("executor", executor)
builder.add_node("generator", generator)
builder.add_node("critic", critic)
builder.add_node("refiner", refiner)
builder.add_node("finalizer", finalizer)

builder.add_edge(START, "planner")
builder.add_edge("planner", "executor")
builder.add_conditional_edges(
    "executor", 
    should_execute, 
    {"executor": "executor", "generator": "generator"}
)
builder.add_edge("generator", "critic")
builder.add_conditional_edges(
    "critic", 
    should_refine, 
    {"refiner": "refiner", "finalizer": "finalizer"}
)
builder.add_edge("refiner", "critic") # Loop back to critic after refining
builder.add_edge("finalizer", END)

hybrid_agent = builder.compile()

In [8]:
task = "Analyze the usefulness of argan oil into a plain-language and make it brief"

result = hybrid_agent.invoke({
    "input": task,
    "plan": [],
    "current_step": 0,
    "results": [],
    "iterations": 0,
    "draft": "",
    "critique": ""
})

# FINAL OUTPUT PRINTING
print("\n" + "="*50)
print("📊 HYBRID AGENT EXECUTION SUMMARY")
print("="*50)
print(f"PLAN: {result['plan']}")
print(f"\nINITIAL DRAFT: {result['draft'][:100]}...")
print(f"\nCRITIQUE: {result['critique'][:100]}...")
print(f"\nFINAL OUTPUT:\n{result['final_output']}")
print("="*50)


📋 PLAN CREATED:
  1. **Research Argan Oil**: Gather information on argan oil, including its origins, production process, and common uses in cosmetics and cooking.
  2. **Identify Benefits**: List the key benefits of argan oil, such as moisturizing properties, antioxidant content, and potential health benefits.
  3. **Evaluate Uses**: Analyze how argan oil is used in everyday products (like skin care and hair care) and its nutritional value when used in cooking.
  4. **Summarize Findings**: Write a brief summary that highlights the main points about argan oil's usefulness in simple language.
  5. **Review and Edit**: Go through the summary to ensure clarity and conciseness, making any necessary adjustments for better understanding.

⚙️ Executing: 1. **Research Argan Oil**: Gather information on argan oil, including its origins, production process, and common uses in cosmetics and cooking.
Step 1 result: ### Argan Oil: Overview

**Origins:**
Argan oil is derived from the nuts of the arg